# 0 - IMPORTS

In [55]:
import tensorflow as tf
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')

# 1 - LOAD DATASET

In [3]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

# 2 - DATA TRANSFORMATION

In [11]:
scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)

In [23]:
# reshape target variable to matrix format "N lines x 1 column" 
y = y.reshape(-1, 1)

# apply one hot encoder to target variable
ct = ColumnTransformer(
    transformers=[
        ("Country",        # Just a name
         OneHotEncoder(), # The transformer class
         [0]            # The column(s) to be applied on.
         )
    ], remainder='passthrough'
)

y = ct.fit_transform(y)
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

# 3 - CREATE TRAIN & TEST DATASET

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [27]:
X_train.shape

(105, 4)

In [28]:
X_test.shape

(45, 4)

# 4 - CREATE TENSORFLOW MODEL

![x_and_y](img/neural_network.png)

## 4.1 Create input layer

In [30]:
input_layer = X.shape[1]
print('input_layer contains {} neurons.'.format(input_layer))

input_layer contains 4 neurons.


## 4.2 Create hidden layer

In [31]:
hidden_layer = int(np.ceil((X.shape[1] + y.shape[1]) / 2))
print('hidden_layer contains {} neurons.'.format(hidden_layer))

hidden_layer contains 4 neurons.


## 4.3 Create output layer

In [32]:
output_layer = y.shape[1]
print('hidden_layer contains {} neurons.'.format(output_layer))

hidden_layer contains 3 neurons.


## 4.4 Create Weights Tensor Variables

In [39]:
W = {'hidden': tf.Variable(tf.random_normal([input_layer, hidden_layer])),
     'output': tf.Variable(tf.random_normal([hidden_layer, output_layer]))}

## 4.5 Create Bias Tensor Variables 

In [40]:
b = {'hidden': tf.Variable(tf.random_normal([hidden_layer])),
     'output': tf.Variable(tf.random_normal([output_layer]))}

## 4.6 Create PlaceHolders Tensor Variables 

In [36]:
xph = tf.placeholder('float', [None, input_layer])
yph = tf.placeholder('float', [None, output_layer])

## 4.7 Create Tensorflow Model

In [37]:
# define multilayer perceptron function
def mlp(x, W, bias):
    input_layer_to_hidden_layer = tf.add(tf.matmul(x, W['hidden']), bias['hidden'])
    hidden_layer_activation     = tf.nn.relu(input_layer_to_hidden_layer)
    output_layer                = tf.add(tf.matmul(hidden_layer_activation, W['output']), b['output'])
    return output_layer

In [43]:
model = mlp(xph, W, b)

## 4.8 Create Model Optimizer

In [45]:
model_error     = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = yph))
model_optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(model_error)

# 5 - TRAINING MODEL

## 5.1 - Define Batch Structure

In [46]:
batch_size  = 8 # 8 records to train and optmize model
batch_cycle = int(len(X_train) / batch_size) # number of cicles to train all records
print( 'We have {} cicles with {} records each.'.format(batch_cycle, batch_size) )

We have 13 cicles with 8 records each.


In [33]:
# create array of batch with predict variables
X_batches = np.array_split(X_train, batch_cycle)
X_batches

[array([[-0.29484182, -0.8191665 ,  0.25096731,  0.13322594],
        [-0.41600969, -1.51337539,  0.02350449, -0.12972   ],
        [ 1.2803405 ,  0.33784833,  1.10395287,  1.44795564],
        [ 1.03800476,  0.10644536,  0.36469871,  0.26469891],
        [ 1.03800476,  0.10644536,  0.53529583,  0.39617188],
        [-0.7795133 ,  0.80065426, -1.3412724 , -1.31297673],
        [-1.02184904, -1.74477836, -0.26082403, -0.26119297],
        [-1.50652052,  1.26346019, -1.56873522, -1.31297673],
        [-0.05250608, -0.8191665 ,  0.76275864,  0.92206376]]),
 array([[ 0.4321654 , -0.58776353,  0.59216153,  0.79059079],
        [-0.90068117,  1.72626612, -1.2844067 , -1.18150376],
        [ 1.15917263,  0.33784833,  1.21768427,  1.44795564],
        [ 0.31099753, -1.05056946,  1.04708716,  0.26469891],
        [-0.7795133 ,  1.03205722, -1.2844067 , -1.31297673],
        [-0.05250608, -1.05056946,  0.1372359 ,  0.00175297],
        [-0.17367395, -0.58776353,  0.42156442,  0.13322594],
      

## 5.2 - Model Training

In [60]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(5000):
        mean_error = 0.0
        batch_cycle = int(len(X_train) / batch_size)
        X_batches = np.array_split(X_train, batch_cycle)
        y_batches = np.array_split(y_train, batch_cycle)
        for i in range(batch_cycle):
            X_batch, y_batch = X_batches[i], y_batches[i]
            _, cost = sess.run([model_optimizer, model_error], feed_dict = {xph: X_batch, yph: y_batch})
            mean_error += cost / batch_cycle
        if epoch % 500 == 0:
            print('Epoch: ' + str((epoch + 1)) + ' error: ' + str(mean_error))
    W_final, b_final = sess.run([W, b])

Epoch: 1 error: 1.5941788095694318
Epoch: 501 error: 0.5234812715878854
Epoch: 1001 error: 0.22008495433972433
Epoch: 1501 error: 0.1038204706632174
Epoch: 2001 error: 0.06434985794700109
Epoch: 2501 error: 0.05062771130066652
Epoch: 3001 error: 0.04367676828629695
Epoch: 3501 error: 0.03942242205644456
Epoch: 4001 error: 0.036492079111317605
Epoch: 4501 error: 0.034304259918057


## 5.3 - Final Weights

In [61]:
W_final

{'hidden': array([[-1.2923219 ,  0.29243767, -0.4963156 , -1.015388  ],
        [-0.3444296 ,  0.16575554, -0.408272  ,  1.0258651 ],
        [-0.3726473 , -0.57454437,  3.1076303 , -2.9574142 ],
        [ 1.3198701 , -0.7447929 ,  1.1177372 , -1.3508388 ]],
       dtype=float32),
 'output': array([[-0.10608529,  0.93869287,  0.84095085],
        [-2.0976825 ,  1.5011766 , -2.6750572 ],
        [-3.1631365 , -0.09710413,  3.282309  ],
        [ 3.5655735 , -0.9623636 , -2.94165   ]], dtype=float32)}

In [62]:
b_final

{'hidden': array([ 1.4578539 ,  2.2436147 , -0.82143253,  0.03312217], dtype=float32),
 'output': array([-2.7112691 , -0.60244083, -0.08208413], dtype=float32)}

# 6 - PREDICTIONS

In [63]:
# predictions
model_predictions = mlp(xph, W_final, b_final)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(model_predictions, feed_dict = {xph: X_test})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))

## 6.1 - Output Layer Results

In [64]:
r1

array([[-12.991252  ,   0.69306797,  10.6043215 ],
       [ 21.84841   ,  -1.7801383 , -33.691284  ],
       [ -5.8798614 ,   2.0134401 ,  -2.442431  ],
       [ 22.791592  ,  -1.9917789 , -34.758163  ],
       [ -5.588908  ,   3.7103179 ,  -0.04260123],
       [ 19.453487  ,  -0.56365734, -29.351967  ],
       [ -5.8762603 ,   3.5091035 ,   0.9477972 ],
       [-10.328619  ,   2.8474247 ,   8.992704  ],
       [ -9.229098  ,   2.6666365 ,   5.471111  ],
       [-11.2662325 ,   1.4946244 ,   7.992957  ],
       [ -6.141271  ,   3.6850145 ,   1.7955183 ],
       [-11.84668   ,   1.2218566 ,   5.452527  ],
       [ 19.434114  ,  -0.21691436, -28.460035  ],
       [ -3.7111812 ,   4.2032647 ,  -5.0556984 ],
       [ -4.3649135 ,   2.7894187 ,  -3.397544  ],
       [ -3.1699004 ,   3.4970105 ,  -4.726263  ],
       [ -8.028805  ,   2.5892947 ,   2.964099  ],
       [ 19.694609  ,  -1.7622857 , -32.184864  ],
       [-10.059749  ,   1.9522116 ,   7.9607344 ],
       [ -3.053021  ,   4.23434

## 6.2 - Softmax Output Layer

In [65]:
r2

array([[5.65654329e-11, 4.96107241e-05, 9.99950409e-01],
       [1.00000000e+00, 5.47333637e-11, 7.57559250e-25],
       [3.68815672e-04, 9.88158524e-01, 1.14727207e-02],
       [1.00000000e+00, 1.72472054e-11, 1.01498338e-25],
       [8.93907127e-05, 9.77000713e-01, 2.29098797e-02],
       [1.00000000e+00, 2.02611616e-09, 6.36884290e-22],
       [7.79213951e-05, 9.28257167e-01, 7.16649890e-02],
       [4.05439327e-09, 2.13899068e-03, 9.97860968e-01],
       [3.89272628e-07, 5.70828468e-02, 9.42916751e-01],
       [4.31704583e-09, 1.50368304e-03, 9.98496294e-01],
       [4.69188089e-05, 8.68657291e-01, 1.31295756e-01],
       [3.02538083e-08, 1.43341795e-02, 9.85665739e-01],
       [1.00000000e+00, 2.92190627e-09, 1.58429129e-21],
       [3.65258078e-04, 9.99539614e-01, 9.52102346e-05],
       [7.79260066e-04, 9.97170508e-01, 2.05024495e-03],
       [1.27036555e-03, 9.98461723e-01, 2.67922500e-04],
       [9.96813924e-06, 4.07376617e-01, 5.92613459e-01],
       [1.00000000e+00, 4.80163

## 6.3 - Argmax Output Layer

In [66]:
r3

array([2, 0, 1, 0, 1, 0, 1, 2, 2, 2, 1, 2, 0, 1, 1, 1, 2, 0, 2, 1, 2, 2,
       0, 2, 0, 0, 0, 0, 1, 2, 0, 2, 2, 2, 1, 0, 2, 1, 0, 0, 2, 2, 1, 1,
       0], dtype=int64)

## 6.4 - Accuracy Score

In [67]:
y_test2 = np.argmax(y_test, 1)
y_test2

array([2, 0, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 1, 1, 0, 2, 1, 2, 2,
       0, 2, 0, 0, 0, 0, 1, 2, 0, 2, 2, 2, 1, 0, 2, 1, 0, 0, 2, 2, 1, 1,
       0], dtype=int64)

In [68]:
model_accuracy_score = accuracy_score(y_test2, r3)
model_accuracy_score

0.9555555555555556